<h2><span style="color:#05AC4B; font-weight: 900">Importação de Bibliotecas</span></h2>

In [ ]:
# Início do cronômetro
import time
from time import strftime
start = time.time()
inicio = time.strftime("%H:%M:%S")
# Query 
from __future__ import division
import pyodbc, sys, socket
# Web Scrappnig
import requests
from bs4 import BeautifulSoup
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
from re import sub
# Json
import json
# Limpar Output
from IPython.display import clear_output
# Decodificar 
import unicodedata
# Verificar a semelhança
from difflib import SequenceMatcher

<h2><span style="color:#05AC4B; font-weight: 900">Query para buscar produtos do departamento 18</span></h2>

In [ ]:
arg=socket.gethostname()
cnxn = pyodbc.connect('DRIVER={DBMAKER};SERVER=' + arg + ';DATABASE=COTRIPALDB_SIMPLUS_BASE;UID=SYSADM;PWD=')
cursor = cnxn.cursor()

sql=("select a.gtin , a.codigo_cotripal, cast(a.descricao as varchar(250)) as nome, a.id_erp_departamento, a.id_erp_setor, a.id_erp_item from produto as a where a.id_erp_departamento = 018 and a.descricao not like '%VV%' ORDER BY IDX;")


cursor.execute(sql)

rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()

<h2><span style="color:#05AC4B; font-weight: 900">Declaração de Variáveis</span></h2>

In [ ]:
infos = []

founded = False
nh = 0
case = 0
rech = 0
commersul = 0
sulmatre = 0

# Declaração de variáveis globais
global founded, nh, case, rech, commersul, sulmatre

# Declaração de dataframes
notFound = pd.DataFrame({'gtin':[], 'cod_cotripal':[], 'nome_produto':[], 'id_erp_departamento':[], 'id_erp_setor':[], 'id_erp_item': [], 'info_extra':[]})
mclFound = pd.DataFrame({'gtin':[], 'cod_cotripal':[], 'nome_produto':[], 'id_erp_departamento':[], 'id_erp_setor':[], 'id_erp_item': []})
notFoundOEM = pd.DataFrame({'gtin':[], 'cod_cotripal':[], 'nome_produto':[], 'id_erp_departamento':[], 'id_erp_setor':[], 'id_erp_item': [], 'info_extra':[]})

<h2><span style="color:#05AC4B; font-weight: 900">Funções de tratamento</span></h2>

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">contains_number</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String recebida como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Caso encontre algum número na string, retorna True, se não, retorna False</span></li>
</ul>

In [ ]:
def contains_number(string):
    return any(char.isdigit() for char in string)

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">str_normalized</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String como parâmetro como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a string sem acentos gráficos</span></li>
</ul>

In [ ]:
def str_normalized(string):
    normalized = unicodedata.normalize('NFD', string)
    return normalized.encode('ascii', 'ignore').decode('utf8').casefold()

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">similaridade</span><span style="color:#333; font-weight: 900">(string, string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe duas Strings como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Envia cada sring para a função <span style="color:#005FDF; font-weight: 400">str_normalized</span> e armazena o retorno</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna um valor entre 0(zero) e 1(um) com o percentual de semelhança entre as duas strings</span></li>
</ul>

In [ ]:
def similaridade(str1, str2):
    str1 = str_normalized(str1)
    str2 = str_normalized(str2)
    return SequenceMatcher(None, str1, str2).ratio()

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">clear_str</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">A variável "remover" contém os caracteres a serem removidos da string</span></li>
<li><span style="color:#333333; font-weight: 900">Verifica se a string é uma url, caso não seja adiciona ":" na lista de caracteres a serem removidos</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a String sem os caracteres da lista</span></li>
</ul>

In [ ]:
def clear_str(text):
    remover = ['\n', ' ', ',', '\t']
    if 'http' not in text:
        remover.append(':')
    for rm in remover:
        text = text.replace(rm,'')
    return text

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">remove_param</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Identifica a posição do "?" na url</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a url sem oos parâmetros</span></li>
</ul>

In [ ]:
def remove_param(text):
    separator = re.search(r'\?', text)
    indice = text.find(separator.group())
    text = text[:indice]
    return text

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">find_images</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<p><span style="color:#333333; font-weight: 900">Utilizada para localizar as imagens no site da New Holland e no site da Case</span></p>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Identifica o marcador '"image": [' no script</span></li>
<li><span style="color:#333333; font-weight: 900">A partir do primeiro marcado identificado, identifica o marcador "]" no script</span></li>
<li><span style="color:#333333; font-weight: 900">Remove caracteres indesejados com a função <span style="color:#005FDF; font-weight: 400">clear_str</span></span></li>
<li><span style="color:#333333; font-weight: 900">Separa as imagens e remove os parâmetros da url com a função <span style="color:#005FDF; font-weight: 400">remove_param</span></span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a url da imagem</span></li>
</ul>

In [ ]:
def find_images(text):
    separator = re.search(r'"image": \[', text)
    indice = text.find(separator.group())
    indice = indice + len('"image": [')
    text = text[indice:]
    separator = re.search(r'\]', text)
    indice = text.find(separator.group())
    text = text[:indice]
    text = clear_str(text)
    text = text.split('"')
    text = list(filter(None, text))
    text = list(map(remove_param, text))
    return text

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">remove_letters</span><span style="color:#333; font-weight: 900">(string, int)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String e um inteiro como parâmetros</span></li>
<li><span style="color:#333333; font-weight: 900">Inicia um laço de repetição while(enquanto)</span></li>
<li><span style="color:#333333; font-weight: 900">Enquanto o caractere da String no indíce, recebidos como parâmetro, não for um número, remove o mesmo</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a String com o primeiro e último caractere sendo caracteres numéricos</span></li>
</ul>

In [ ]:
def remove_letters(string, index):
    while not string[index].isdigit():
        string = string.replace(string[index], '')
    return string

<h2><span style="color:#008639; font-weight: 900">def </span><span style="color:#005FDF; font-weight: 900">findOEM</span><span style="color:#333; font-weight: 900">(string)</span></h2>
<br>
<ul>
<li><span style="color:#333333; font-weight: 900">Recebe uma String como parâmetro</span></li>
<li><span style="color:#333333; font-weight: 900">Inicia uma variável do tipo lista vazia.</span></li>
<li><span style="color:#333333; font-weight: 900">Localiza o índice do primeiro número encotrado no nome.</span></li>
<li><span style="color:#333333; font-weight: 900">Remove tudo que antecede o primeiro número, pontos e traços da String.</span></li>
<li><span style="color:#333333; font-weight: 900">Separa a String restante pelos espaços e verifica individualmente cada parte se contém número, e se o tamanho é maior que 5.</span></li>
<li><span style="color:#333333; font-weight: 900">Caso o tamanho da String resultante seja maior que 5 e contenha números adiciona na variável oem.</span></li>
<li><span style="color:#333333; font-weight: 900">Retorna a variável oem, podendo ser vazia ou conter um ou mais valores</span></li>
</ul>

In [ ]:
def findOEM(text):
    oem = []
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                oem.append(txt)
    return oem

<h2><span style="color:#05AC4B; font-weight: 900">Funções de Busca</span></h2>

In [ ]:
def search_sulmatre(prod, cod):
    url = 'https://loja.caseih.com.br/search?text='
    headers = {
        "User-Agent" : ""
    }
    
    listOfGlobals = globals()

    for codigo in cod:
        if not founded:
            link = str(url + codigo)
            req = requests.get(link, headers=headers)
            if req.status_code != 200:
                continue
            site = BeautifulSoup(req.text, "html.parser")
            posts = [p.find("a") for p in site.find_all("header", class_="entry-header")]
            if posts:
                for post in posts:
                    try:
                        foundedOEMs = findOEM(post.get_text())
                    except:
                        continue
                    for oem in foundedOEMs:
                        if not founded:
                            if similaridade(oem, codigo) > .9:
                                listOfGlobals['founded'] = True
                                listOfGlobals['sulmatre'] += 1
                                link = post['href']
                                req = requests.get(link, headers=headers)
                                if req.status_code != 200:
                                    continue
                                site = BeautifulSoup(req.text, "html.parser")
                                nome = site.find("h1", class_="product_title").get_text()
                                images = [img['src'] for img in site.find_all("img") if codigo in img['src']]
                                itemInfo = {
                                    "gtin" : prod[0],
                                    "codigo_cotripal" : prod[1],
                                    "id_erp_departamento" : prod[3],
                                    "id_erp_setor" : prod[4],
                                    "id_erp_item" : prod[5],
                                    "descricao_erp": prod[2],
                                    "nome" : nome,
                                    "imagens" : images,
                                    "lista_variavel": [],
                                    "link" : link
                                }
                                keys = [tr.find('th', class_='woocommerce-product-attributes-item__label').get_text() for tr in site.find_all('tr', class_='woocommerce-product-attributes-item') if tr.find('th', class_='woocommerce-product-attributes-item__label').get_text() != 'Código Sulmatre']
                                values = [str(tr.find('td', class_='woocommerce-product-attributes-item__value').get_text()).replace('\n', '') for tr in site.find_all('tr', class_='woocommerce-product-attributes-item') if tr.find('th', class_='woocommerce-product-attributes-item__label').get_text() != 'Código Sulmatre']
                                for i, key in enumerate(keys):
                                    itemInfo["lista_variavel"].append({key : values[i]})
                                infos.append(itemInfo)
    if not founded:
        notFoundOEM.loc[len(notFoundOEM)] = [prod[0], prod[1], prod[2], prod[3], prod[4], prod[5], 'produto não encontrado em nenhum site']
    return

In [ ]:
def search_commersul(prod, cod):
    url = 'https://www.commersul.com.br/?post_type=product&s='
    headers = {
        "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
    }
    
    listOfGlobals = globals()

    for codigo in cod:
        if not founded:
            link = str(url + codigo)
            req = requests.get(link, headers=headers)
            if req.status_code != 200:
                continue
            site = BeautifulSoup(req.text, "html.parser")
            naoEncontrado = site.find("p", class_='woocommerce-no-products-found')
            listOfGlobals['site'] = site
            if naoEncontrado:
                continue
            else:
                resultados = site.find_all("li", class_="product")
                if resultados:
                    for resultado in resultados:
                        if not founded:
                            foundedOEMs = findOEM(f' {resultado.find("h2", class_="woocommerce-loop-product__title").get_text()}')
                            for oem in foundedOEMs:
                                if similaridade(oem, codigo) > .90:
                                    listOfGlobals['founded'] = True
                                    listOfGlobals['commersul'] += 1
                                    link = resultado.find('a', class_='woocommerce-loop-product__link')['href']
                                    req = requests.get(link, headers=headers)
                                    if req.status_code != 200:
                                        continue
                                    site = BeautifulSoup(req.text, "html.parser")
                nome = site.title.get_text()
                itemInfo = {
                    "gtin" : prod[0],
                    "codigo_cotripal" : prod[1],
                    "id_erp_departamento" : prod[3],
                    "id_erp_setor" : prod[4],
                    "id_erp_item" : prod[5],
                    "descricao_erp": prod[2],
                    "nome" : nome,
                    "imagens" : [],
                    "lista_variavel": [],
                    "link" : link
                }
                div = site.find('div', class_='woocommerce-Tabs-panel--description')
                infos = [v.get_text() for v in div.find_all('li')]
                keys = [li.split(':')[0] for li in infos if li != infos[-1]]
                values = [li.split(':')[1] for li in infos if li != infos[-1]]
                for i, key in enumerate(keys):
                    itemInfo["lista_variavel"].append({key : values[i]})
                infos.append(itemInfo)
    if not founded:
        search_sulmatre(prod, cod)
    return

In [ ]:
def search_rech(prod, cod):
    url = 'https://rech.com/busca?q='
    headers = {
        "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
    }
    
    listOfGlobals = globals()

    for codigo in cod:
        if not founded:
            link = str(url + codigo)
            req = requests.get(link, headers=headers)
            if req.status_code != 200:
                continue
            site = BeautifulSoup(req.text, "html.parser")

            naoEncontrado = site.find("h4", class_='searchNotFound-title')
            if naoEncontrado:
                continue
            else:
                resultados = site.find_all("div", class_="y-item-wrapper-showcase")
                if resultados:
                    for resultado in resultados:
                        if not founded:
                            nome = clear_str(resultado.find("h2", class_="produto-nome").get_text())
                            oem = resultado.find("div", class_="produto-mfrPartCode").get_text()
                            oem = oem.replace('Cód.', '')
                            if len(str(findOEM(oem))) > 2:
                                foundedOEM = remove_letters(str(findOEM(oem)), 0)
                                foundedOEM = remove_letters(foundedOEM, -1)
                                clearedOEM = remove_letters(codigo, 0)
                                clearedOEM = remove_letters(clearedOEM, -1)
                                if similaridade(foundedOEM, clearedOEM) > .90:
                                    listOfGlobals['founded'] = True
                                    listOfGlobals['rech'] += 1
                                    link = 'https://rech.com'+resultado.find('link')['href']
                                    req = requests.get(link, headers=headers)
                                    if req.status_code != 200:
                                        continue
                                    site = BeautifulSoup(req.text, "html.parser")
                                    urlImagens = [img['src'] for img in site.find_all('img', class_='thumb') if 'indisponivel' not in img['src']]
                                    itemInfo = {
                                        "gtin" : prod[0],
                                        "codigo_cotripal" : prod[1],
                                        "id_erp_departamento" : prod[3],
                                        "id_erp_setor" : prod[4],
                                        "id_erp_item" : prod[5],
                                        "descricao_erp": prod[2],
                                        "nome" : clear_str(nome),
                                        "imagens" : urlImagens,
                                        "lista_variavel": [],
                                        "link" : link
                                    }
                                    table = site.find('table', class_='table-specifications')
                                    keys = [clear_str(x.get_text()) for x in table.find_all('th')]
                                    values = [x.get_text() for x in table.find_all('td')]
                                    values[-1] = values[-1].replace('\t', '')
                                    values[-1] = values[-1].replace('\n', '')
                                    values[-1] = values[-1].replace('  ', '')
                                    values[-1] = values[-1].replace('(1 ATÉ 99999) -', ',')
                                    values[-1] = values[-1].replace(' ,VER MAIS', '')
                                    for i, key in enumerate(keys):
                                        itemInfo["lista_variavel"].append({key : values[i]})
                                        infos.append(itemInfo)
    if not founded:
        search_commersul(prod, cod)
    return

In [ ]:
def search_case(prod, cod):
    url = 'https://loja.caseih.com.br/search?text='
    headers = {
        "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
    }
    
    listOfGlobals = globals()

    for codigo in cod:
        if not founded:
            link = str(url + codigo)
            req = requests.get(link, headers=headers)
            if req.status_code != 200:
                continue
            site = BeautifulSoup(req.text, "html.parser")

            naoEncontrado = site.find("div", {"id": "product-not-found"})
            if naoEncontrado:
                continue
            else:
                resultados = site.find_all("div", class_="product-card-wrapper")
                if resultados:
                    for resultado in resultados:
                        if not founded:
                            nome = resultado.find("h4", class_="product-card__details__name")
                            for oemEncotrado in findOEM(nome.get_text()):
                                if similaridade(oemEncotrado, codigo) > .95:
                                    listOfGlobals['founded'] = True
                                    listOfGlobals['case'] += 1
                                    link = 'https://loja.caseih.com.br'+nome.find("a")['href']
                                    req = requests.get(link, headers=headers)
                                    if req.status_code != 200:
                                        continue
                                    site = BeautifulSoup(req.text, "html.parser")
                                    main = site.find("main", class_="pdp-page")
                                    urlImagens = find_images(str(main.find("script")))
                                    itemInfo = {
                                        "gtin" : prod[0],
                                        "codigo_cotripal" : prod[1],
                                        "id_erp_departamento" : prod[3],
                                        "id_erp_setor" : prod[4],
                                        "id_erp_item" : prod[5],
                                        "descricao_erp": prod[2],
                                        "nome" : nome.get_text(),
                                        "imagens" : urlImagens,
                                        "lista_variavel": [],
                                        "link" : link
                                    }
                                    keys = [clear_str(x.get_text()) for x in site.find_all('h4', class_='product-specifications-item__title')]
                                    values = [x.get_text() for x in site.find_all('strong', class_='product-specifications-item__value')]
                                    for i, key in enumerate(keys):
                                        itemInfo["lista_variavel"].append({key : values[i]})
                                    infos.append(itemInfo)
    if not founded:
        search_rech(prod, cod)
    return

In [ ]:
def search_newHolland(prod, cod):
    url = 'https://loja.newhollandag.com.br/search?text='
    headers = {
        "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
    }
    
    listOfGlobals = globals()

    for codigo in cod:
        if not founded:
            link = str(url + codigo)
            req = requests.get(link, headers=headers)
            if req.status_code != 200:
                continue
            site = BeautifulSoup(req.text, "html.parser")

            naoEncontrado = site.find("div", {"id": "product-not-found"})
            if naoEncontrado:
                continue
            else:
                resultados = site.find_all("div", class_="product-card-wrapper")
                if resultados:
                    for resultado in resultados:
                        if not founded:
                            nome = resultado.find("h4", class_="product-card__details__name")
                            for oemEncotrado in findOEM(nome.get_text()):
                                if similaridade(oemEncotrado, codigo) > .95:
                                    listOfGlobals['founded'] = True
                                    listOfGlobals['nh'] += 1
                                    link = 'https://loja.newhollandag.com.br'+nome.find("a")['href']
                                    req = requests.get(link, headers=headers)
                                    if req.status_code != 200:
                                        continue
                                    site = BeautifulSoup(req.text, "html.parser")
                                    main = site.find("main", class_="pdp-page")
                                    urlImagens = find_images(str(main.find("script")))
                                    itemInfo = {
                                        "gtin" : prod[0],
                                        "codigo_cotripal" : prod[1],
                                        "id_erp_departamento" : prod[3],
                                        "id_erp_setor" : prod[4],
                                        "id_erp_item" : prod[5],
                                        "descricao_erp": prod[2],
                                        "nome" : nome.get_text(),
                                        "imagens" : urlImagens,
                                        "lista_variavel": [],
                                        "link" : link
                                    }
                                    keys = [clear_str(x.get_text()) for x in site.find_all('h4', class_='product-specifications-item__title')]
                                    values = [x.get_text() for x in site.find_all('strong', class_='product-specifications-item__value')]
                                    for i, key in enumerate(keys):
                                        itemInfo["lista_variavel"].append({key : values[i]})
                                    infos.append(itemInfo)
    if not founded:
        search_case(prod, cod)
    return

In [ ]:
mclFound = pd.DataFrame({'gtin':[], 'cod_cotripal':[], 'nome_produto':[], 'id_erp_departamento':[], 'id_erp_setor':[], 'id_erp_item': []})

<p><span style="color:#333333; font-weight: 900">Se o nome do produto conter <span style="color:#D40000; font-weight: 900">'MCL'</span> salva as informações do produto no dataframe <span style="color:#005FDF; font-weight: 400">mclFound</span></span></p>
<p><span style="color:#333333; font-weight: 900">Se o nome do produto não conter números salva as informações do produto no dataframe <span style="color:#005FDF; font-weight: 400">notFoundOEM</span></span></p>
<p><span style="color:#333333; font-weight: 900">Verifica se é possível obter algum código no nome, caso encontre inicia as buscas, se não encontrar salva as informações do produto no dataframe <span style="color:#005FDF; font-weight: 400">notFoundOEM</span></span></p>

In [ ]:
for i, row in enumerate(rows):
    if 'MCL' in row[2]:
        mclFound.loc[len(mclFound)] = [row[0], row[1], row[2], row[3], row[4], row[5]] # Possivelmente 14 produtos
        continue
    if not contains_number(row[2]):
        notFoundOEM.loc[len(notFoundOEM)] = [row[0], row[1], row[2], row[3], row[4], row[5], 'Não contém número']
        continue
    cod = findOEM(row[2])
    if cod:
        search_newHolland(row, cod)
    else:
        notFoundOEM.loc[len(notFoundOEM)] = [row[0], row[1], row[2], row[3], row[4], row[5], 'Código encontrado não é OEM']
    
    end = time.time()
    clear_output(wait=True)
    print('|- Carregando... {:->12}'.format('|'))
    print('|{:->28}'.format('|'))
    print('|- {:06.2f}% Concluido {:->8}'.format(((i + 1)/len(rows))*100, '|'))
    print('|- {:03} Produtos restantes {:->3}'.format(len(rows)-(i + 1), '|'))
    print('|{:->28}'.format('|'))
    print('{:-<11} timer: {:07.2f} -|'.format('|', (end-start)/60))
    print('|{:->28}'.format('|'))

In [ ]:
# Parada do cronômetro
fim = time.strftime("%H:%M:%S")